# 什么是Data Leakage
这里的Data Leakage跟其他场合说的数据安全数据泄漏完全不一样。从字面上理解，我们说的Data Leakage不是数据泄漏，而是因果关系的纰漏，是由于数据准备过程中出现的失误，使模型沿着有纰漏的，甚至是颠倒的因果关系进行预测，但得到极好的预测结果。

Data Leakage是数据科学家需要了解的最重要问题之一。如果不知道如何防止Data Leakage，那么Data Leakage会频繁出现，并且会以最微妙和危险的方式破坏你的模型。具体而言，Data Leakage导致模型看起来准确，直到开始使用模型做出决策，然后模型变得非常不准确。

有两种主要的泄漏类型：Leaky Predictors和Leaky Validation Strategies。

## Leaky Predictors
当预测因子包含做出预测时无法获得的数据时，就会发生这种情况。

比如，想预测谁会得肺炎。原始数据的前几行可能如下所示：

|got_pneumonia|age|weight|male|took_antibiotic_medicine|...|
|:-:|:-:|:----:|:-:|:-:|
|False|65|100|False|False|...|
|False|72|130|True|False|...|
|True|58|100|False|True|...|

人们为了治愈通常会服用抗生素药物。所有，数据列之间有很强的因果关系。`took_antibiotic_medicine`值得改变通常由`got_pneumonia`决定。这是target leakage。
   
模型会发现，所有`took_antibiotic_medicine`为`False`的人，都没有肺炎。验证数据属于同一个来源，因此该模式将在验证中重复，并且该模型将具有极好的验证（或交叉验证）分数。但随后在现实世界中部署时，该模型将非常不准确。

为了防止这种类型的Data Leakage，应该排除得到目标模型之后会更新（或创建）的任何变量。因为当我们使用这个模型做出新的预测时，这些数据将会变成不可用数据。

## Leaky Validation Strategy
如果不小心将验证数据与训练数据区分开来，则会发生不同类型的泄漏。例如，如果在调用`train_test_split`之前，对整个数据集运行预处理（如为缺少的值拟合Imputer）。模型验证旨在衡量模型如何处理之前未考虑过的数据。这样导致的结果就是，验证分数很高，但是在实际数据中表现很差。

## 防止Leaky Predictors
没有统一的方案来解决这种泄漏。处理之前，需要对数据有一定的了解、特定案例的检查及相关常识。

Leaky Predictors经常与目标有很高的统计相关性。要记住这两点：
- 寻找与目标统计相关的列
- 如果建立的模型非常准确，那么可能存在这类问题

## 防止Leaky Validation Strategies
如果验证数据来自简单的`Train-Test`划分，要排除对验证集做的任何类型的`fitting`，包括预处理过程。

# 示例
我们将使用一个关于信用卡申请的小数据集，我们将建立一个模型来预测哪些申请会被接受（存储在一个名为`card`的变量中）。

先看一下我们的数据集：

In [2]:
import pandas as pd

data = pd.read_csv('input/AER_credit_card_data.csv', true_values=['yes'], false_values=['no'])

print(data.head())

   card  reports       age  income     share  expenditure  owner  selfemp  \
0  True        0  37.66667  4.5200  0.033270   124.983300   True    False   
1  True        0  33.25000  2.4200  0.005217     9.854167  False    False   
2  True        0  33.66667  4.5000  0.004156    15.000000   True    False   
3  True        0  30.50000  2.5400  0.065214   137.869200  False    False   
4  True        0  32.16667  9.7867  0.067051   546.503300   True    False   

   dependents  months  majorcards  active  
0           3      54           1      12  
1           3      34           1      13  
2           4      58           1       5  
3           0      25           1       7  
4           2      64           1       5  


In [3]:
print(data.shape)

(1319, 12)


这是一个小数据集，为了保证模型质量，应该使用交叉验证。

In [4]:
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

y = data.card
X = data.drop(['card'], axis=1)

modeling_pipeline = make_pipeline(RandomForestClassifier()) # 这里没有对数据进行预处理，但使用Pipeline依然是很好的选择
cv_scores = cross_val_score(modeling_pipeline, X, y, scoring='accuracy')
print("Cross-val accuracy: %f" % cv_scores.mean())

Cross-val accuracy: 0.979526


凭借经验，你会发现很难找到准确率为98％的模型。因此，我们应该更仔细地检查数据，以查看它是否存在Data Leakage。

数据描述：
- card: Dummy variable, 1 if application for credit card accepted, 0 if not
- reports: Number of major derogatory reports
- age: Age n years plus twelfths of a year
- income: Yearly income (divided by 10,000)
- share: Ratio of monthly credit card expenditure to yearly income
- expenditure: Average monthly credit card expenditure
- owner: 1 if owns their home, 0 if rent
- selfempl: 1 if self employed, 0 if not.
- dependents: 1 + number of dependents
- months: Months living at current address
- majorcards: Number of major credit cards held
- active: Number of active credit accounts

一些变量看起来很可疑。例如，支出(expenditure)是指的是在这张卡片上，还是在申请之前使用的卡片上？

针对这种情况，基本的数据比较会非常有用。

In [8]:
expenditures_cardholders = data.expenditure[data.card]
expenditures_noncardholders = data.expenditure[~data.card]

print('Fraction of those who received a card with no expenditures: %.2f' \
      %(( expenditures_cardholders == 0).mean()))
print('Fraction of those who have not received a card with no expenditures: %.2f' \
      %((expenditures_noncardholders == 0).mean()))

Fraction of those who received a card with no expenditures: 0.02
Fraction of those who have not received a card with no expenditures: 1.00


可以看出，每个`card == False`的人都没有支出，与此同时，只有2%的`card == True`的人没有支出。这种情况下，可能存在数据泄漏。

由于`share`由`expenditure`决定，所以也应该一并排除。变量`active`、`majorcards`不太明确，从描述看，应该排除。大多数情况下，这样做会比较安全。

再次运行模型。

In [9]:
potential_leaks = ['expenditure', 'share', 'active', 'majorcards']
X2 = X.drop(potential_leaks, axis=1)
cv_scores = cross_val_score(modeling_pipeline, X2, y, scoring='accuracy')
print("Cross-val accuracy: %f" %cv_scores.mean())

Cross-val accuracy: 0.807433


## 结论
- 仔细分离`Train-Test`数据集是第一步，管道可以帮助实现这种分离。
- Leaking predictors是很常见的问题，并且很难被发现。结合谨慎、数据常识、数据探索，可以很好地识别 Leaking predictors。

# 参考
- [Data Leakage](https://blog.csdn.net/jiandanjinxin/article/details/54633475)